In [1]:
import cirq
import numpy as np
import random

import sympy

Finding large cuts for a graph using variational circuits. We take a graph with $n$ vertices and $m$ edges. For every vertex, we add a qubit.

In [2]:
# Vertices n and Edges m.
n = 6
m = 8
qubits = cirq.LineQubit.range(n)

We randomly place the edges between vertices. This completes the graph that we have to cut. The variational approach follows a standard form called the QAOA (Quantum Approximate Optimisation Algorithm). The function that has to be maximised is $\frac{1}{2}(1 - \sum_{<i,j>}Z_i Z_j)$ where the sum is over the links. Whenever a link is cut, the cost function increases by 1 if the two sets of vertices seperated by the cut are assigned orthogonal states $\ket{0}$ and $\ket{1}$.

In [4]:
random.seed(0)
links = []
size_links = np.size(links)/2
while size_links < m:
    a = random.choice(qubits)
    b = random.choice(qubits)
    if a != b:
        if [a,b] not in links and [b,a] not in links:
            links.append([a,b])
            size_links += 1
print(links)

[[cirq.LineQubit(0), cirq.LineQubit(2)], [cirq.LineQubit(4), cirq.LineQubit(3)], [cirq.LineQubit(3), cirq.LineQubit(2)], [cirq.LineQubit(4), cirq.LineQubit(1)], [cirq.LineQubit(2), cirq.LineQubit(1)], [cirq.LineQubit(0), cirq.LineQubit(4)], [cirq.LineQubit(2), cirq.LineQubit(4)], [cirq.LineQubit(5), cirq.LineQubit(4)]]


In [6]:
def qaoa_max_cut(gamma_params : np.ndarray, beta_params : np.ndarray, layers) -> cirq.Circuit:
    '''This function produces the variational circuit with layers and parameters given'''
    max_cut_circuit = cirq.Circuit()
    max_cut_circuit.append(cirq.H.on_each(qubits))
    for i in range(layers):
        for [a,b] in links:
            max_cut_circuit.append(cirq.ZZPowGate(exponent=gamma_params[i]).on(a,b))
        max_cut_circuit.append(cirq.XPowGate(exponent = beta_params[i]).on_each(qubits))
    return max_cut_circuit

The circuit is shown below for a single layer.

In [7]:
gamma = sympy.Symbol('gamma')
beta = sympy.Symbol('beta')
max_cut_circuit = qaoa_max_cut(gamma_params=[gamma], beta_params=[beta], layers = 1)
max_cut_circuit.append(cirq.measure(qubits))
print(max_cut_circuit)

                     ┌────────────────┐   ┌────────────────┐
0: ───H───ZZ───────────────────────────────────────ZZ──────────X^beta─────────────────────────M───
          │                                        │                                          │
1: ───H───┼───────────────────ZZ───────────ZZ──────┼───────────X^beta─────────────────────────M───
          │                   │            │       │                                          │
2: ───H───ZZ^gamma────ZZ──────┼────────────ZZ^gamma┼───────────ZZ─────────X^beta──────────────M───
                      │       │                    │           │                              │
3: ───H───ZZ──────────ZZ^gamma┼────────────X^beta──┼───────────┼──────────────────────────────M───
          │                   │                    │           │                              │
4: ───H───ZZ^gamma────────────ZZ^gamma─────────────ZZ^gamma────ZZ^gamma───ZZ─────────X^beta───M───
                                                            

In [8]:
def cost_function_calculator(measurement, links):
    '''A cost function calculator for the max cut. It has to be maximised to maximise the number of cuts.'''
    cost = 0
    for [a,b] in links:
        index_a = qubits.index(a)
        index_b = qubits.index(b)
        if measurement[index_a] != measurement[index_b]:
            cost = cost + 1
    return cost

In [9]:
# When there is no cut, each of the qubits have the same state, either 0 or 1.
print(cost_function_calculator([1,1,1,1,1,1], links))

0


In [10]:
sim = cirq.Simulator()
max_cut_circuit = qaoa_max_cut(gamma_params=[0.75], beta_params=[0.25], layers = 1)
max_cut_circuit.append(cirq.measure(qubits))
result = sim.run(max_cut_circuit, repetitions=5)
measurement = result.measurements['q(0),q(1),q(2),q(3),q(4),q(5)']
print(measurement)
print(measurement[3])


[[1 1 1 0 0 0]
 [0 1 1 0 0 0]
 [1 1 0 0 0 1]
 [1 1 0 1 0 0]
 [1 1 0 1 0 0]]
[1 1 0 1 0 0]


In [11]:
steps = 100
cost = np.zeros((steps,steps), dtype=float)
config = np.zeros((steps,steps), dtype=tuple)
for i in range(steps):
    for j in range(steps):
        beta_val = i*2/steps
        gamma_val = j*2/steps
        sim = cirq.Simulator()
        circuit = qaoa_max_cut(gamma_params=[gamma_val], beta_params=[beta_val], layers = 1)
        circuit.append(cirq.measure(qubits))
        result = sim.run(circuit, repetitions=1)
        measurement = result.measurements['q(0),q(1),q(2),q(3),q(4),q(5)'][0]
        cost[i,j] = cost_function_calculator(measurement, links)
        config[i,j] = measurement


In [14]:
print(f"The max cut estimator is :", np.max(cost))
np.where(cost == np.max(cost))

The max cut estimator is : 7.0


(array([ 0,  1,  3,  3,  4,  4,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  6,
         6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,
         9,  9, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 12, 12, 12, 12,
        12, 12, 12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14, 14,
        14, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 17, 17, 18, 18, 18,
        19, 19, 19, 19, 20, 20, 20, 20, 20, 20, 20, 21, 21, 21, 21, 21, 21,
        21, 22, 22, 22, 23, 23, 23, 23, 23, 24, 24, 24, 25, 25, 25, 26, 26,
        26, 27, 27, 27, 27, 28, 28, 29, 29, 30, 30, 30, 30, 30, 31, 31, 32,
        32, 32, 33, 33, 33, 33, 33, 33, 33, 34, 34, 34, 34, 34, 34, 34, 34,
        35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 37, 37, 37, 37, 37, 38,
        38, 38, 38, 38, 38, 39, 39, 39, 39, 39, 39, 39, 40, 40, 40, 40, 40,
        40, 40, 41, 41, 41, 41, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 44,
        44, 44, 44, 45, 45, 45, 46, 47, 47, 47, 47, 48, 48, 48, 49, 49, 49,
        50, 

The matrix represents the parameter values where this cut was found. Using these we can find a configuration that gives this large cut.

In [17]:
print(f"A configuration with the maximum cut estimator : {cost[0,48]} is : {config[0,48]}")

A configuration with the maximum cut estimator : 7.0 is : [1 1 0 1 0 1]
